In [3]:
%pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 4.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [93]:
import pandas as pd 


import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit


In [94]:
df=pd.read_csv("full_dataset.csv")
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [95]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [96]:
df['isFraud'].value_counts()

isFraud
0    6354407
1       8213
Name: count, dtype: int64

### Since here the categories are in imbalanced number we will use ADASYN to make equal number of categories

In [97]:
categorical_cols=df.select_dtypes(exclude=np.number).columns
categorical_cols

Index(['type', 'nameOrig', 'nameDest'], dtype='object')

In [98]:
for cols in categorical_cols:
    print(cols)
    print(df[cols].value_counts())
    print()
    

type
type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64

nameOrig
nameOrig
C1902386530    3
C363736674     3
C545315117     3
C724452879     3
C1784010646    3
              ..
C98968405      1
C720209255     1
C1567523029    1
C644777639     1
C1280323807    1
Name: count, Length: 6353307, dtype: int64

nameDest
nameDest
C1286084959    113
C985934102     109
C665576141     105
C2083562754    102
C248609774     101
              ... 
M1470027725      1
M1330329251      1
M1784358659      1
M2081431099      1
C2080388513      1
Name: count, Length: 2722362, dtype: int64



In [99]:
type_encoded=pd.DataFrame(pd.get_dummies(df['type']))
type_encoded


,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,False,False,False,True,False
1,False,False,False,True,False
2,False,False,False,False,True
3,False,True,False,False,False
4,False,False,False,True,False
...,...,...,...,...,...
6362615,False,True,False,False,False
6362616,False,False,False,False,True
6362617,False,True,False,False,False
6362618,False,False,False,False,True


In [100]:
df=pd.concat([df,type_encoded],axis=1)
df.drop('type',axis=1,inplace=True)

In [101]:
df

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,1,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0,False,False,False,True,False
1,1,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0,False,False,False,True,False
2,1,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0,False,False,False,False,True
3,1,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0,False,True,False,False,False
4,1,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0,False,True,False,False,False
6362616,743,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0,False,False,False,False,True
6362617,743,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0,False,True,False,False,False
6362618,743,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0,False,False,False,False,True


In [39]:
le=LabelEncoder()
nameDest_encoded=list(le.fit_transform(df['nameDest']))
df.drop('nameDest',axis=1,inplace=True)
df['nameDest_encoded']=nameDest_encoded



In [40]:
le=LabelEncoder()
nameOrig_encoded=list(le.fit_transform(df['nameOrig']))
df.drop('nameOrig',axis=1,inplace=True)
df['nameOrig_encoded']=nameOrig_encoded


In [41]:
df.dtypes


step                  int64
amount              float64
oldbalanceOrg       float64
newbalanceOrig      float64
oldbalanceDest      float64
newbalanceDest      float64
isFraud               int64
isFlaggedFraud        int64
CASH_IN                bool
CASH_OUT               bool
DEBIT                  bool
PAYMENT                bool
TRANSFER               bool
nameDest_encoded      int64
nameOrig_encoded      int64
dtype: object

In [42]:
df

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,nameDest_encoded,nameOrig_encoded
0,1,9839.64,170136.00,160296.36,0.00,0.00,0,0,False,False,False,True,False,1662094,757869
1,1,1864.28,21249.00,19384.72,0.00,0.00,0,0,False,False,False,True,False,1733924,2188998
2,1,181.00,181.00,0.00,0.00,0.00,1,0,False,False,False,False,True,439685,1002156
3,1,181.00,181.00,0.00,21182.00,0.00,1,0,False,True,False,False,False,391696,5828262
4,1,11668.14,41554.00,29885.86,0.00,0.00,0,0,False,False,False,True,False,828919,3445981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,339682.13,0.00,0.00,339682.13,1,0,False,True,False,False,False,505863,5651847
6362616,743,6311409.28,6311409.28,0.00,0.00,0.00,1,0,False,False,False,False,True,260949,1737278
6362617,743,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,0,False,True,False,False,False,108224,533958
6362618,743,850002.52,850002.52,0.00,0.00,0.00,1,0,False,False,False,False,True,319713,2252932


In [54]:



# Separate features and target
X = df.drop('isFraud', axis=1)
Y = df['isFraud']



# Apply ADASYN to the training data
adasyn = ADASYN(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X, Y)

# Combine the resampled data into a DataFrame
resampled_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['isFraud'])], axis=1)

# Verify the balance
print(resampled_df['isFraud'].value_counts(normalize=True))


isFraud
0    0.500063
1    0.499937
Name: proportion, dtype: float64


In [55]:
np.unique(y_resampled,return_counts=True)

(array([0, 1]), array([6354407, 6352806]))

In [56]:
resampled_df

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,nameDest_encoded,nameOrig_encoded,isFraud
0,1,9839.640000,170136.000000,160296.36,0.000000e+00,0.000000e+00,0,False,False,False,True,False,1662094,757869,0
1,1,1864.280000,21249.000000,19384.72,0.000000e+00,0.000000e+00,0,False,False,False,True,False,1733924,2188998,0
2,1,181.000000,181.000000,0.00,0.000000e+00,0.000000e+00,0,False,False,False,False,True,439685,1002156,1
3,1,181.000000,181.000000,0.00,2.118200e+04,0.000000e+00,0,False,True,False,False,False,391696,5828262,1
4,1,11668.140000,41554.000000,29885.86,0.000000e+00,0.000000e+00,0,False,False,False,True,False,828919,3445981,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12707208,507,250096.314650,250096.314650,0.00,7.411580e+06,7.661677e+06,0,False,True,False,False,False,76975,1302450,1
12707209,723,833431.258768,833431.258768,0.00,6.563005e+06,7.396436e+06,0,False,True,False,False,False,503727,963982,1
12707210,505,245467.109161,245467.109161,0.00,7.418537e+06,7.664004e+06,0,False,True,False,False,False,73444,1305407,1
12707211,656,630559.068202,630559.068202,0.00,6.839858e+06,7.470417e+06,0,False,True,False,False,False,367199,1059409,1


### Using the Stratified Sampling technqiue

In [81]:
12707213*0.004


50828.852

In [83]:
sample_size =0.004

# Set up stratified sampling
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=sample_size, random_state=42)

# Extract the indices for the stratified sample
for train_index, sample_index in stratified_split.split(resampled_df, resampled_df['isFraud']):
    stratified_sample_df = resampled_df.iloc[sample_index]

# Verify the balance in the sampled dataset
print(stratified_sample_df['isFraud'].value_counts(normalize=True))
print(resampled_df['isFraud'].value_counts(normalize=True))

isFraud
0    0.500069
1    0.499931
Name: proportion, dtype: float64
isFraud
0    0.500063
1    0.499937
Name: proportion, dtype: float64


In [85]:
stratified_sample_df['isFraud'].value_counts()

isFraud
0    25418
1    25411
Name: count, dtype: int64

In [86]:
final_df=stratified_sample_df.copy(deep=True)
final_df

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,nameDest_encoded,nameOrig_encoded,isFraud
12623652,563,18417.240320,18417.240320,0.00,1.240395e+04,1.551259e+04,0,False,True,False,False,True,209519,2572822,1
11169036,419,346354.615847,346354.615847,0.00,0.000000e+00,3.463546e+05,0,False,True,False,False,False,403995,6311127,1
2910667,229,291590.310000,6049.000000,0.00,2.497772e+04,3.165680e+05,0,False,True,False,False,False,482160,3861560,0
9295942,418,66777.824054,66777.824054,0.00,0.000000e+00,0.000000e+00,0,False,False,False,False,True,351115,1371643,1
5483046,379,678.130000,45082.000000,44403.87,0.000000e+00,0.000000e+00,0,False,False,False,True,False,2184529,5689466,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8580662,244,717917.637444,717917.637444,0.00,1.174964e+07,1.246756e+07,0,False,True,False,False,False,370174,4423416,1
5424913,378,302.460000,792.000000,489.54,0.000000e+00,0.000000e+00,0,False,False,False,True,False,2151612,1769882,0
2358220,190,11327.270000,10303.000000,0.00,0.000000e+00,0.000000e+00,0,False,False,False,True,False,1204550,6093817,0
11064978,512,264334.474702,264334.474702,0.00,0.000000e+00,0.000000e+00,0,False,False,False,False,True,422565,1356128,1


In [89]:
final_df.reset_index(drop=True,inplace=True)
final_df

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,nameDest_encoded,nameOrig_encoded,isFraud
0,563,18417.240320,18417.240320,0.00,1.240395e+04,1.551259e+04,0,False,True,False,False,True,209519,2572822,1
1,419,346354.615847,346354.615847,0.00,0.000000e+00,3.463546e+05,0,False,True,False,False,False,403995,6311127,1
2,229,291590.310000,6049.000000,0.00,2.497772e+04,3.165680e+05,0,False,True,False,False,False,482160,3861560,0
3,418,66777.824054,66777.824054,0.00,0.000000e+00,0.000000e+00,0,False,False,False,False,True,351115,1371643,1
4,379,678.130000,45082.000000,44403.87,0.000000e+00,0.000000e+00,0,False,False,False,True,False,2184529,5689466,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50824,244,717917.637444,717917.637444,0.00,1.174964e+07,1.246756e+07,0,False,True,False,False,False,370174,4423416,1
50825,378,302.460000,792.000000,489.54,0.000000e+00,0.000000e+00,0,False,False,False,True,False,2151612,1769882,0
50826,190,11327.270000,10303.000000,0.00,0.000000e+00,0.000000e+00,0,False,False,False,True,False,1204550,6093817,0
50827,512,264334.474702,264334.474702,0.00,0.000000e+00,0.000000e+00,0,False,False,False,False,True,422565,1356128,1


In [90]:
# final_df.to_csv('sampled_final_df.csv',index=False)

In [92]:
sampled_final_df=pd.read_csv('sampled_final_df.csv')
sampled_final_df

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,nameDest_encoded,nameOrig_encoded,isFraud
0,563,18417.240320,18417.240320,0.00,1.240395e+04,1.551259e+04,0,False,True,False,False,True,209519,2572822,1
1,419,346354.615847,346354.615847,0.00,0.000000e+00,3.463546e+05,0,False,True,False,False,False,403995,6311127,1
2,229,291590.310000,6049.000000,0.00,2.497772e+04,3.165680e+05,0,False,True,False,False,False,482160,3861560,0
3,418,66777.824054,66777.824054,0.00,0.000000e+00,0.000000e+00,0,False,False,False,False,True,351115,1371643,1
4,379,678.130000,45082.000000,44403.87,0.000000e+00,0.000000e+00,0,False,False,False,True,False,2184529,5689466,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50824,244,717917.637444,717917.637444,0.00,1.174964e+07,1.246756e+07,0,False,True,False,False,False,370174,4423416,1
50825,378,302.460000,792.000000,489.54,0.000000e+00,0.000000e+00,0,False,False,False,True,False,2151612,1769882,0
50826,190,11327.270000,10303.000000,0.00,0.000000e+00,0.000000e+00,0,False,False,False,True,False,1204550,6093817,0
50827,512,264334.474702,264334.474702,0.00,0.000000e+00,0.000000e+00,0,False,False,False,False,True,422565,1356128,1
